In [1]:
pip install PyPDF2 nltk transformers torch


Note: you may need to restart the kernel to use updated packages.


In [3]:
import PyPDF2
import nltk
from transformers import AutoTokenizer, AutoModel
import torch

nltk.download('punkt')

# Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Chunk the Text into 100-token chunks while preserving sentence boundaries
def chunk_text(text, max_tokens=100):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = nltk.word_tokenize(sentence)
        if current_tokens + len(sentence_tokens) > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = sentence_tokens
            current_tokens = len(sentence_tokens)
        else:
            current_chunk.extend(sentence_tokens)
            current_tokens += len(sentence_tokens)

    if current_chunk:
        chunks.append(' '.join(current_chunk))
        
    return chunks

# Step 3: Embed the Chunks using SBERT
def embed_chunks(chunks):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

    embeddings = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    
    return embeddings

# Function to process each textbook
def process_textbook(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text)
    embeddings = embed_chunks(chunks)
    return chunks, embeddings

# Main code to process multiple textbooks
pdf_paths = ['Genesis.pdf', 'For-the-Win.pdf', 'Crime-and-Punishment-.pdf']

all_chunks = []
all_embeddings = []

for pdf_path in pdf_paths:
    chunks, embeddings = process_textbook(pdf_path)
    all_chunks.append(chunks)
    all_embeddings.append(embeddings)

# Print summary of results
for i, pdf_path in enumerate(pdf_paths):
    print(f"Processing {pdf_path}")
    print("Number of chunks:", len(all_chunks[i]))
    print("First chunk:", all_chunks[i][0])
    print("First embedding:", all_embeddings[i][0])
    print("\n")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing Genesis.pdf
Number of chunks: 1865
First chunk: GENESIS THE NEW AGE SERIES BOOK 1 NICOLETTE FULLERCONTENTS New Age Series Genesis Epilogue Next in the Series About the AuthorCopyright ( C ) 2023 Nicolette Fuller Layout design and Copyright ( C ) 2023 by Next Chapter Published 2023 by Next Chapter Edited by Graham ( Fading Street Services ) Cover art by Lordan June Pinote This book is a work of fiction . Names , characters , places , and incidents are the product of the author's imagination or are used fictitiously .
First embedding: [-9.22030285e-02 -2.71565523e-02  1.19216070e-01 -2.34746058e-02
 -1.36343703e-01  3.20764743e-02 -1.87892452e-01 -6.98150173e-02
 -2.39683148e-02  1.20928302e-01  1.12080993e-03  1.11789271e-01
 -1.54976360e-02 -1.16953351e-01 -2.07575515e-01 -3.82153280e-02
 -1.36479750e-01  2.63315830e-02  2.03476354e-01 -5.85764833e-02
 -4.83172201e-02  1.30340233e-02  5.06383479e-02  2.28723977e-02
  5.31177074e-02  5.93501516e-03 -3.48909572e-02  4.83379094

In [2]:
pip install scikit-learn openai pymilvus

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import PyPDF2
import nltk
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.mixture import GaussianMixture
from openai import OpenAI

nltk.download('punkt')

# Configure OpenAI API
openai_api_key = 'sk-proj-l9SYpydEKHt8KEkmL5ndT3BlbkFJvOtUrurp5oegpvflblF8'
client = OpenAI(api_key=openai_api_key)

# # Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# Define the Milvus schema
fields = [
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="metadata", dtype=DataType.JSON)
]
schema = CollectionSchema(fields, description="RAPTOR index")

Create a collection
collection = Collection(name="raptor_index", schema=schema)

# Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Chunk the Text into 100-token chunks while preserving sentence boundaries
def chunk_text(text, max_tokens=100):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = nltk.word_tokenize(sentence)
        if current_tokens + len(sentence_tokens) > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = sentence_tokens
            current_tokens = len(sentence_tokens)
        else:
            current_chunk.extend(sentence_tokens)
            current_tokens += len(sentence_tokens)

    if current_chunk:
        chunks.append(' '.join(current_chunk))
        
    return chunks

# Embed the Chunks using SBERT
def embed_chunks(chunks):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

    embeddings = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    
    return embeddings

# Cluster embeddings using GMM
def cluster_embeddings(embeddings, n_clusters=5):
    gmm = GaussianMixture(n_clusters, covariance_type='tied').fit(embeddings)
    soft_labels = gmm.predict_proba(embeddings)
    return soft_labels, gmm.means_

# Summarize clusters using GPT-3.5-turbo
def summarize_clusters(chunks, soft_labels, n_clusters):
    summaries = []
    for i in range(n_clusters):
        cluster_texts = [chunks[j] for j in range(len(chunks)) if soft_labels[j][i] > 0.5]
        summary_input = ' '.join(cluster_texts[:5])  # Summarize the first 5 chunks for simplicity
        response = client.chat.completions.create(
            messages=[
                {"role": "user", "content": f"Summarize the following text: {summary_input}"}
            ],
            model="gpt-3.5-turbo",
        )
        summary = response.choices[0].message['content'].strip()
        summaries.append(summary)
    return summaries

# Recursive RAPTOR indexing
def raptor_index(chunks, embeddings, depth=3, current_depth=0):
    if current_depth >= depth:
        return
    
    n_clusters = min(5, len(embeddings))  # Adjust the number of clusters based on data size
    soft_labels, cluster_means = cluster_embeddings(embeddings, n_clusters)
    summaries = summarize_clusters(chunks, soft_labels, n_clusters)
    
    # Re-embed the summaries
    summary_embeddings = embed_chunks(summaries)
    
#     # Store the current level in Milvus
    for i, summary_embedding in enumerate(summary_embeddings):
        metadata = {"level": current_depth, "summary": summaries[i]}
        collection.insert([[summary_embedding.tolist()], [metadata]])
    
    # Recurse for the next level
    for i in range(n_clusters):
        cluster_chunks = [chunks[j] for j in range(len(chunks)) if soft_labels[j][i] > 0.5]
        cluster_embeddings_list = [embeddings[j] for j in range(len(embeddings)) if soft_labels[j][i] > 0.5]
        if len(cluster_chunks) > 1:  # Proceed only if there is enough data to cluster
            raptor_index(cluster_chunks, cluster_embeddings_list, depth, current_depth + 1)

# Process each textbook and create RAPTOR index
def process_textbook(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text)
    embeddings = embed_chunks(chunks)
    raptor_index(chunks, embeddings)

# Main code to process multiple textbooks
pdf_paths = ['Genesis.pdf', 'For-the-Win.pdf', 'Crime-and-Punishment-.pdf']

for pdf_path in pdf_paths:
    process_textbook(pdf_path)

print("RAPTOR indexing completed.")


In [ ]:
import os
import PyPDF2
import nltk
from transformers import AutoTokenizer, AutoModel, DPRContextEncoder, DPRQuestionEncoder
import torch
from sklearn.mixture import GaussianMixture
from openai import OpenAI
from rank_bm25 import BM25Okapi
from nltk.corpus import wordnet
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType

nltk.download('punkt')
nltk.download('wordnet')

# Configure OpenAI API
openai_api_key = 'sk-proj-y8KxRkLyMsuCBsm8qFv6T3BlbkFJ3BjalSYNacoV6GcUwhkG'
client = OpenAI(api_key=openai_api_key)

# Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# Define the Milvus schema
fields = [
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="metadata", dtype=DataType.JSON)
]
schema = CollectionSchema(fields, description="RAPTOR index")

# Create a collection
collection = Collection(name="raptor_index", schema=schema)

# Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Chunk the Text into 100-token chunks while preserving sentence boundaries
def chunk_text(text, max_tokens=100):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = nltk.word_tokenize(sentence)
        if current_tokens + len(sentence_tokens) > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = sentence_tokens
            current_tokens = len(sentence_tokens)
        else:
            current_chunk.extend(sentence_tokens)
            current_tokens += len(sentence_tokens)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Embed the Chunks using SBERT
def embed_chunks(chunks):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

    embeddings = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

    return embeddings

# Cluster embeddings using GMM
def cluster_embeddings(embeddings, n_clusters=5):
    gmm = GaussianMixture(n_clusters, covariance_type='tied').fit(embeddings)
    soft_labels = gmm.predict_proba(embeddings)
    return soft_labels, gmm.means_

# Summarize clusters using GPT-3.5-turbo
def summarize_clusters(chunks, soft_labels, n_clusters):
    summaries = []
    for i in range(n_clusters):
        cluster_texts = [chunks[j] for j in range(len(chunks)) if soft_labels[j][i] > 0.5]
        summary_input = ' '.join(cluster_texts[:5])  # Summarize the first 5 chunks for simplicity
        response = client.chat.completions.create(
            messages=[
                {"role": "user", "content": f"Summarize the following text: {summary_input}"}
            ],
            model="gpt-3.5-turbo",
        )
        summary = response.choices[0].message['content'].strip()
        summaries.append(summary)
    return summaries

# Recursive RAPTOR indexing
def raptor_index(chunks, embeddings, depth=3, current_depth=0):
    if current_depth >= depth:
        return

    n_clusters = min(5, len(embeddings))  # Adjust the number of clusters based on data size
    soft_labels, cluster_means = cluster_embeddings(embeddings, n_clusters)
    summaries = summarize_clusters(chunks, soft_labels, n_clusters)

    # Re-embed the summaries
    summary_embeddings = embed_chunks(summaries)

    # Store the current level in Milvus
    for i, summary_embedding in enumerate(summary_embeddings):
        metadata = {"level": current_depth, "summary": summaries[i]}
        collection.insert([[summary_embedding.tolist()], [metadata]])

    # Recurse for the next level
    for i in range(n_clusters):
        cluster_chunks = [chunks[j] for j in range(len(chunks)) if soft_labels[j][i] > 0.5]
        cluster_embeddings_list = [embeddings[j] for j in range(len(embeddings)) if soft_labels[j][i] > 0.5]
        if len(cluster_chunks) > 1:  # Proceed only if there is enough data to cluster
            raptor_index(cluster_chunks, cluster_embeddings_list, depth, current_depth + 1)

# Query Expansion
def expand_query(query):
    synonyms = set()
    for word in nltk.word_tokenize(query):
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
    return ' '.join(synonyms)

# Hybrid Retrieval
def hybrid_retrieval(query, chunks, embeddings):
    expanded_query = expand_query(query)
    bm25 = BM25Okapi([nltk.word_tokenize(chunk) for chunk in chunks])
    bm25_scores = bm25.get_scores(nltk.word_tokenize(expanded_query))

    tokenizer = AutoTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
    model = AutoModel.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

    inputs = tokenizer(query, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        query_embedding = model(**inputs).pooler_output.squeeze().numpy()

    bert_scores = [torch.cosine_similarity(torch.tensor(query_embedding), torch.tensor(embedding)).item() for embedding in embeddings]

    combined_scores = [(bm25_score + bert_score) / 2 for bm25_score, bert_score in zip(bm25_scores, bert_scores)]

    ranked_indices = sorted(range(len(combined_scores)), key=lambda i: combined_scores[i], reverse=True)
    ranked_chunks = [chunks[i] for i in ranked_indices]

    return ranked_chunks[:5]

# Re-ranking retrieved data
def rerank_retrieved_data(query, retrieved_chunks):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": f"Re-rank the following texts based on their relevance to the query: {query}. Texts: {' '.join(retrieved_chunks)}"}
        ],
        model="gpt-3.5-turbo",
    )
    ranked_texts = response.choices[0].message['content'].strip()
    return ranked_texts.split('\n')

# Process each textbook and create RAPTOR index
def process_textbook(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text)
    embeddings = embed_chunks(chunks)
    raptor_index(chunks, embeddings)

# Main code to process multiple textbooks
pdf_paths = ['Genesis.pdf', 'For-the-Win.pdf', 'Crime-and-Punishment-.pdf']

for pdf_path in pdf_paths:
    process_textbook(pdf_path)

# Example query
query = "What is the impact of social media on youth?"
chunks = chunk_text(extract_text_from_pdf('Genesis.pdf'))
embeddings = embed_chunks(chunks)
retrieved_chunks = hybrid_retrieval(query, chunks, embeddings)
ranked_chunks = rerank_retrieved_data(query, retrieved_chunks)

print("Top relevant chunks:")
for chunk in ranked_chunks:
    print(chunk)



In [ ]:
import os
import PyPDF2
import nltk
from transformers import AutoTokenizer, AutoModel, DPRContextEncoder, DPRQuestionEncoder
import torch
from sklearn.mixture import GaussianMixture
from openai import OpenAI
from rank_bm25 import BM25Okapi
from nltk.corpus import wordnet
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType

nltk.download('punkt')
nltk.download('wordnet')

# Configure OpenAI API
openai_api_key = 'sk-proj-y8KxRkLyMsuCBsm8qFv6T3BlbkFJ3BjalSYNacoV6GcUwhkG'
client = OpenAI(api_key=openai_api_key)

# Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# Define the Milvus schema
fields = [
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="metadata", dtype=DataType.JSON)
]
schema = CollectionSchema(fields, description="RAPTOR index")

# Create a collection
collection = Collection(name="raptor_index", schema=schema)

# Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Chunk the Text into 100-token chunks while preserving sentence boundaries
def chunk_text(text, max_tokens=100):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    for sentence in sentences:
        sentence_tokens = nltk.word_tokenize(sentence)
        if current_tokens + len(sentence_tokens) > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = sentence_tokens
            current_tokens = len(sentence_tokens)
        else:
            current_chunk.extend(sentence_tokens)
            current_tokens += len(sentence_tokens)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Embed the Chunks using SBERT
def embed_chunks(chunks):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

    embeddings = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

    return embeddings

# Cluster embeddings using GMM
def cluster_embeddings(embeddings, n_clusters=5):
    gmm = GaussianMixture(n_clusters, covariance_type='tied').fit(embeddings)
    soft_labels = gmm.predict_proba(embeddings)
    return soft_labels, gmm.means_

# Summarize clusters using GPT-3.5-turbo
def summarize_clusters(chunks, soft_labels, n_clusters):
    summaries = []
    for i in range(n_clusters):
        cluster_texts = [chunks[j] for j in range(len(chunks)) if soft_labels[j][i] > 0.5]
        summary_input = ' '.join(cluster_texts[:5])  # Summarize the first 5 chunks for simplicity
        response = client.chat.completions.create(
            messages=[
                {"role": "user", "content": f"Summarize the following text: {summary_input}"}
            ],
            model="gpt-3.5-turbo",
        )
        summary = response.choices[0].message['content'].strip()
        summaries.append(summary)
    return summaries

# Recursive RAPTOR indexing
def raptor_index(chunks, embeddings, depth=3, current_depth=0):
    if current_depth >= depth:
        return

    n_clusters = min(5, len(embeddings))  # Adjust the number of clusters based on data size
    soft_labels, cluster_means = cluster_embeddings(embeddings, n_clusters)
    summaries = summarize_clusters(chunks, soft_labels, n_clusters)

    # Re-embed the summaries
    summary_embeddings = embed_chunks(summaries)

    # Store the current level in Milvus
    for i, summary_embedding in enumerate(summary_embeddings):
        metadata = {"level": current_depth, "summary": summaries[i]}
        collection.insert([[summary_embedding.tolist()], [metadata]])

    # Recurse for the next level
    for i in range(n_clusters):
        cluster_chunks = [chunks[j] for j in range(len(chunks)) if soft_labels[j][i] > 0.5]
        cluster_embeddings_list = [embeddings[j] for j in range(len(embeddings)) if soft_labels[j][i] > 0.5]
        if len(cluster_chunks) > 1:  # Proceed only if there is enough data to cluster
            raptor_index(cluster_chunks, cluster_embeddings_list, depth, current_depth + 1)

# Query Expansion
def expand_query(query):
    synonyms = set()
    for word in nltk.word_tokenize(query):
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
    return ' '.join(synonyms)

# Hybrid Retrieval
def hybrid_retrieval(query, chunks, embeddings):
    expanded_query = expand_query(query)
    bm25 = BM25Okapi([nltk.word_tokenize(chunk) for chunk in chunks])
    bm25_scores = bm25.get_scores(nltk.word_tokenize(expanded_query))

    tokenizer = AutoTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
    model = AutoModel.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

    inputs = tokenizer(query, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        query_embedding = model(**inputs).pooler_output.squeeze().numpy()

    bert_scores = [torch.cosine_similarity(torch.tensor(query_embedding), torch.tensor(embedding)).item() for embedding in embeddings]

    combined_scores = [(bm25_score + bert_score) / 2 for bm25_score, bert_score in zip(bm25_scores, bert_scores)]

    ranked_indices = sorted(range(len(combined_scores)), key=lambda i: combined_scores[i], reverse=True)
    ranked_chunks = [chunks[i] for i in ranked_indices]

    return ranked_chunks[:5]

# Re-ranking retrieved data
def rerank_retrieved_data(query, retrieved_chunks):
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": f"Re-rank the following texts based on their relevance to the query: {query}. Texts: {' '.join(retrieved_chunks)}"}
        ],
        model="gpt-3.5-turbo",
    )
    ranked_texts = response.choices[0].message['content'].strip()
    return ranked_texts.split('\n')

# Question Answering using LLM
def answer_question(query, ranked_chunks):
    context = ' '.join(ranked_chunks)
    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": f"Answer the following question based on the given context: {query}. Context: {context}"}
        ],
        model="gpt-3.5-turbo",
    )
    answer = response.choices[0].message['content'].strip()
    return answer

# Process each textbook and create RAPTOR index
def process_textbook(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text)
    embeddings = embed_chunks(chunks)
    raptor_index(chunks, embeddings)

# Main code to process multiple textbooks
pdf_paths = ['Genesis.pdf', 'For-the-Win.pdf', 'Crime-and-Punishment-.pdf']

for pdf_path in pdf_paths:
    process_textbook(pdf_path)

# Example query
query = "What is the impact of social media on youth?"
chunks = chunk_text(extract_text_from_pdf('Genesis.pdf'))
embeddings = embed_chunks(chunks)
retrieved_chunks = hybrid_retrieval(query, chunks, embeddings)
ranked_chunks = rerank_retrieved_data(query, retrieved_chunks)
answer = answer_question(query, ranked_chunks)

print("Top relevant chunks:")
for chunk in ranked_chunks:
    print(chunk)

print("\nAnswer to the question:")
print(answer)
